In [10]:
import pandas as pd
from transformers import BertTokenizer, BertModel
import torch
import numpy as np
from torch import nn
from torch.optim import Adam
from tqdm import tqdm
import re

In [17]:
df = pd.read_csv('../../dataframe.csv')
df = df.dropna(subset=['text', 'is_offensive'])
df['is_offensive'] = df['is_offensive'].astype('int')
df['target'] = df['target'].map({'INSULT': 0, 'RACIST': 1, 'SEXIST': 2, 'PROFANITY': 3, 'OTHER': 4})
df = df[~(((df['is_offensive'] == 0) & (df['target'] != 4)) | df['text'].isnull() | (df['is_offensive'] == 1) & (df['target'] == 4))]
df = df.drop(['id','target'], axis=1)
df.reset_index(drop=True, inplace=True)
for i in range(len(df)):
    if len(df['text'][i]) == 1:
        #delete row
        df = df.drop(i)
df.reset_index(drop=True, inplace=True)
df

,text,is_offensive
0,çürük dişli,1
1,Bu adamın islama ve müslümanlara verdiği zarar...,1
2,erkekler zora gelmez,1
3,Utanmazın götüne kazık sokmuşlar bu tıkırtı ne...,1
4,otomasyon< sistemlerine= doğrudan bağlanabilir,0
...,...,...
12406,uyuma taklidi yapan tehlikeli bir hayvanın göz...,0
12407,yolda at kavga eden üç oğlan çocuğu görür,0
12408,sizin köpeklerinizin burnu bile daha iyi koku ...,0
12409,hayalleri gerçek etmek için birisinin delilik ...,0


In [18]:
def preprocess(text):
    text = text.lower()
    text = re.sub(r'[^\w\s]', '', text)
    text = re.sub(r'\s+', ' ', text)
    text = re.sub('[0-9]', '', text)
    return text

for i in range(len(df)):
    df['text'][i] = preprocess(df['text'][i])

C:\Users\hasan\AppData\Local\Temp/ipykernel_14988/655321863.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['text'][i] = preprocess(df['text'][i])


In [19]:
tokenizer = BertTokenizer.from_pretrained("dbmdz/bert-base-turkish-cased")

class Dataset(torch.utils.data.Dataset):
    
    def __init__(self, df):

        self.labels = [label for label in df['is_offensive']]
        self.texts = [tokenizer(text, 
                               padding='max_length', max_length = 512, truncation=True,
                                return_tensors="pt") for text in df['text']]

    def classes(self):
        return self.labels

    def __len__(self):
        return len(self.labels)

    def get_batch_labels(self, idx):
        # Fetch a batch of labels
        return np.array(self.labels[idx])

    def get_batch_texts(self, idx):
        # Fetch a batch of inputs
        return self.texts[idx]

    def __getitem__(self, idx):

        batch_texts = self.get_batch_texts(idx)
        batch_y = self.get_batch_labels(idx)

        return batch_texts, batch_y

In [20]:
class BertClassifier(nn.Module):

    def __init__(self, dropout=0.5):

        super(BertClassifier, self).__init__()

        self.bert = BertModel.from_pretrained("dbmdz/bert-base-turkish-cased")
        self.dropout = nn.Dropout(dropout)
        self.linear = nn.Linear(768, 2)
        self.relu = nn.ReLU()

    def forward(self, input_id, mask):

        _, pooled_output = self.bert(input_ids= input_id, attention_mask=mask, return_dict=False)
        dropout_output = self.dropout(pooled_output)
        linear_output = self.linear(dropout_output)
        final_layer = self.relu(linear_output)

        return final_layer

In [21]:
def train(model, train_data, val_data, learning_rate, epochs):

    train, val = Dataset(train_data), Dataset(val_data)

    train_dataloader = torch.utils.data.DataLoader(train, batch_size=2, shuffle=True)
    val_dataloader = torch.utils.data.DataLoader(val, batch_size=2)

    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

    criterion = nn.CrossEntropyLoss()
    optimizer = Adam(model.parameters(), lr= learning_rate)

    if use_cuda:
        model = model.cuda()
        criterion = criterion.cuda()

    for epoch_num in range(epochs):

        total_acc_train = 0
        total_loss_train = 0

        for train_input, train_label in tqdm(train_dataloader):

            train_label = train_label.to(device)
            mask = train_input['attention_mask'].to(device)
            input_id = train_input['input_ids'].squeeze(1).to(device)

            output = model(input_id, mask)
            
            batch_loss = criterion(output, train_label.long())
            total_loss_train += batch_loss.item()
            
            acc = (output.argmax(dim=1) == train_label).sum().item()
            total_acc_train += acc

            model.zero_grad()
            batch_loss.backward()
            optimizer.step()
        
        total_acc_val = 0
        total_loss_val = 0

        with torch.no_grad():

            for val_input, val_label in val_dataloader:

                val_label = val_label.to(device)
                mask = val_input['attention_mask'].to(device)
                input_id = val_input['input_ids'].squeeze(1).to(device)

                output = model(input_id, mask)

                batch_loss = criterion(output, val_label.long())
                total_loss_val += batch_loss.item()
                
                acc = (output.argmax(dim=1) == val_label).sum().item()
                total_acc_val += acc
        
        print(
            f'Epochs: {epoch_num + 1} | Train Loss: {total_loss_train / len(train_data): .3f} \
            | Train Accuracy: {total_acc_train / len(train_data): .3f} \
            | Val Loss: {total_loss_val / len(val_data): .3f} \
            | Val Accuracy: {total_acc_val / len(val_data): .3f}')

In [22]:
df

,text,is_offensive
0,çürük dişli,1
1,bu adamın islama ve müslümanlara verdiği zarar...,1
2,erkekler zora gelmez,1
3,utanmazın götüne kazık sokmuşlar bu tıkırtı ne...,1
4,otomasyon sistemlerine doğrudan bağlanabilir,0
...,...,...
12406,uyuma taklidi yapan tehlikeli bir hayvanın göz...,0
12407,yolda at kavga eden üç oğlan çocuğu görür,0
12408,sizin köpeklerinizin burnu bile daha iyi koku ...,0
12409,hayalleri gerçek etmek için birisinin delilik ...,0


In [27]:
np.random.seed(42)
df_train = df[:-1]
df_val = df[-1:]

EPOCHS = 2
model = BertClassifier()
LR = 1e-6
              
train(model, df_train, df_val, LR, EPOCHS)

Some weights of the model checkpoint at dbmdz/bert-base-turkish-cased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
100%|██████████| 6205/6205 [44:17<00:00,  2.33it/s]


Epochs: 1 | Train Loss:  0.136             | Train Accuracy:  0.886             | Val Loss:  0.596             | Val Accuracy:  1.000


100%|██████████| 6205/6205 [44:21<00:00,  2.33it/s]


Epochs: 2 | Train Loss:  0.052             | Train Accuracy:  0.965             | Val Loss:  0.325             | Val Accuracy:  1.000


In [8]:
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
EPOCHS = 2
LR = 1e-6

# make 10 folds cross validation separating train test validation sets
kf = KFold(n_splits=10, shuffle=True, random_state=42)
i = 1
for train_index, test_index in kf.split(df):
    print(f'This is the {i}th fold')
    df_train, df_test = df.iloc[train_index], df.iloc[test_index]
    df_train, df_val = train_test_split(df_train, test_size=0.1, random_state=42)
    model = BertClassifier()
    train(model, df_train, df_val, LR, EPOCHS)
    i+=1

This is the 1th fold


Some weights of the model checkpoint at dbmdz/bert-base-turkish-cased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
100%|██████████| 5026/5026 [35:59<00:00,  2.33it/s]


Epochs: 1 | Train Loss:  0.160             | Train Accuracy:  0.855             | Val Loss:  0.094             | Val Accuracy:  0.936


100%|██████████| 5026/5026 [35:59<00:00,  2.33it/s]


Epochs: 2 | Train Loss:  0.053             | Train Accuracy:  0.966             | Val Loss:  0.074             | Val Accuracy:  0.952
This is the 1th fold


...

Epochs: 2 | Train Loss:  0.044             | Train Accuracy:  0.971             | Val Loss:  0.044             | Val Accuracy:  0.973
This is the 1th fold


Some weights of the model checkpoint at dbmdz/bert-base-turkish-cased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
100%|██████████| 5027/5027 [35:51<00:00,  2.34it/s]


Epochs: 1 | Train Loss:  0.161             | Train Accuracy:  0.853             | Val Loss:  0.071             | Val Accuracy:  0.949


100%|██████████| 5027/5027 [35:51<00:00,  2.34it/s]


Epochs: 2 | Train Loss:  0.045             | Train Accuracy:  0.969             | Val Loss:  0.050             | Val Accuracy:  0.967


In [9]:
# 1. fold 1. epoch -> Train Loss:  0.160              | Train Accuracy:  0.855              | Val Loss:  0.094              | Val Accuracy:  0.936
# 1. fold 2. epoch -> Train Loss:  0.053              | Train Accuracy:  0.966              | Val Loss:  0.074              | Val Accuracy:  0.952

# 2. fold 1. epoch -> Train Loss:  0.142              | Train Accuracy:  0.884              | Val Loss:  0.059              | Val Accuracy:  0.963
# 2. fold 2. epoch -> Train Loss:  0.049              | Train Accuracy:  0.969              | Val Loss:  0.050              | Val Accuracy:  0.962
 
# 3. fold 1. epoch -> Train Loss:  0.138              | Train Accuracy:  0.883              | Val Loss:  0.062              | Val Accuracy:  0.965
# 3. fold 2. epoch -> Train Loss:  0.041              | Train Accuracy:  0.975              | Val Loss:  0.045              | Val Accuracy:  0.971

# 4. fold 1. epoch -> Train Loss:  0.152              | Train Accuracy:  0.870              | Val Loss:  0.066              | Val Accuracy:  0.953  
# 4. fold 2. epoch -> Train Loss:  0.048              | Train Accuracy:  0.968              | Val Loss:  0.058              | Val Accuracy:  0.958  

# 5. fold 1. epoch -> Train Loss:  0.140              | Train Accuracy:  0.877              | Val Loss:  0.061              | Val Accuracy:  0.962   
# 5. fold 2. epoch -> Train Loss:  0.046              | Train Accuracy:  0.970              | Val Loss:  0.046              | Val Accuracy:  0.970

# 6. fold 1. epoch -> Train Loss:  0.147              | Train Accuracy:  0.892              | Val Loss:  0.056              | Val Accuracy:  0.963  
# 6. fold 2. epoch -> Train Loss:  0.041              | Train Accuracy:  0.973              | Val Loss:  0.044              | Val Accuracy:  0.972 

# 7. fold 1. epoch -> Train Loss:  0.162              | Train Accuracy:  0.850              | Val Loss:  0.059              | Val Accuracy:  0.955  
# 7. fold 2. epoch -> Train Loss:  0.045              | Train Accuracy:  0.970              | Val Loss:  0.046              | Val Accuracy:  0.969

# 8. fold 1. epoch -> Train Loss:  0.156              | Train Accuracy:  0.868              | Val Loss:  0.072              | Val Accuracy:  0.950  
# 8. fold 2. epoch -> Train Loss:  0.052              | Train Accuracy:  0.967              | Val Loss:  0.050              | Val Accuracy:  0.962  

# 9. fold 1. epoch -> Train Loss:  0.160              | Train Accuracy:  0.866              | Val Loss:  0.060              | Val Accuracy:  0.965  
# 9. fold 2. epoch -> Train Loss:  0.044              | Train Accuracy:  0.971              | Val Loss:  0.044              | Val Accuracy:  0.973 

# 10. fold 1. epoch ->Train Loss:  0.161              | Train Accuracy:  0.853              | Val Loss:  0.071              | Val Accuracy:  0.949
# 10. fold 2. epoch ->Train Loss:  0.045              | Train Accuracy:  0.969              | Val Loss:  0.050              | Val Accuracy:  0.967

# print out the average accuracy and loss of 10 folds cross validation
train_acc = [0.966, 0.969, 0.975, 0.968, 0.970, 0.973, 0.970, 0.967, 0.971, 0.971, 0.969]
val_acc = [0.952, 0.962, 0.971, 0.958, 0.970, 0.972, 0.969, 0.962, 0.973, 0.973, 0.967]

print(f'The average accuracy of train set is {np.mean(train_acc)}')
print(f'The average accuracy of validation set is {np.mean(val_acc)}')

The average accuracy of train set is 0.9699090909090908
The average accuracy of validation set is 0.9662727272727274


In [13]:
def evaluate(model, test_data):

    test = Dataset(test_data)

    test_dataloader = torch.utils.data.DataLoader(test, batch_size=2)

    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

    if use_cuda:

        model = model.cuda()

    total_acc_test = 0
    with torch.no_grad():

        for test_input, test_label in test_dataloader:

              test_label = test_label.to(device)
              mask = test_input['attention_mask'].to(device)
              input_id = test_input['input_ids'].squeeze(1).to(device)

              output = model(input_id, mask)

              acc = (output.argmax(dim=1) == test_label).sum().item()
              total_acc_test += acc
    
    print(f'Test Accuracy: {total_acc_test / len(test_data): .3f}')
    
evaluate(model, df_test)

Test Accuracy:  0.962


In [28]:
# try to predict a sentence
def predict(model, sentence):
    
        use_cuda = torch.cuda.is_available()
        device = torch.device("cuda" if use_cuda else "cpu")
    
        if use_cuda:
            model = model.cuda()
    
        with torch.no_grad():
    
            input_id = tokenizer(sentence, padding='max_length', max_length = 512, truncation=True, return_tensors="pt")
            mask = input_id['attention_mask'].to(device)
            input_id = input_id['input_ids'].squeeze(1).to(device)
    
            output = model(input_id, mask)
    
            return output.argmax(dim=1).item()

In [29]:
predict(model, "bence doğudakiler kötü")

1

In [31]:
df_test['text'].iloc[0]

'gürcülerin ne kadar kaba olduğunu bilirim'

In [32]:
true = 0
for i in range(len(df_test.text)):
    result = predict(model, df_test.text.iloc[i])
    if result == df_test.is_offensive.iloc[i]:
        true += 1

acc = true / len(df_test.text)

In [33]:
acc

0.9589371980676329

In [34]:
df_test['is_offensive'].value_counts()

1    891
0    351
Name: is_offensive, dtype: int64

In [30]:
torch.save(model, './models/bert_model_is_offensive.pt')

In [12]:
from sklearn.metrics import f1_score

predicted = []
for i in range(len(df_test.text)):
    predicted.append(predict(model, df_test.text.iloc[i]))
    
f1_score(df_test.is_offensive, predicted, average=None)

0.971815107102593

In [15]:
f1_score(df_test.is_offensive, predicted, average=None)

array([0.92937853, 0.97181511])